In [84]:
import lxml.etree as ET
import hashlib
from templates import templates

In [80]:
class TeiReader():
    
    """ a class to read an process tei-documents"""
    
    def __init__(self, file):
        self.ns_tei = {'tei': "http://www.tei-c.org/ns/1.0"}
        self.ns_xml = {'xml': "http://www.w3.org/XML/1998/namespace"}
        self.file = file
        try:
            self.tree = ET.parse(file)
        except:
            None
        try:
            self.parsed_file = ET.tostring(self.tree, encoding="utf-8")
        except:
            self.parsed_file = "parsing didn't work"
    
    def create_place(self, xml_id="something", text="someplace"):
        
        """ creates a tei:place element with an @xml:id and a child element tei:placeName"""
        
        place = ET.Element("place")
        place.attrib['{http://www.w3.org/XML/1998/namespace}id'] = xml_id
        placeName = ET.Element("placeName")
        placeName.text = text
        place.append(placeName)
        return place
    
    def get_places_elements(self, ids):
        
        """ takes a list of elements with a text node and a @ref attribute and returns a tei:placeList"""
        
        places = []
        for x in ids:
            text = x['text']
            ref = x['ref'][1:]
            place = (text, ref)
            places.append(place)
        place_elements = []
        for text, ref in set(places):
            place = self.create_place(ref, text)
            place_elements.append(place)
        return place_elements
    
    def find_elements(self, tei_element='placeName'):
        
        """ parses a tei:TEI//tei:text element,
        * extracts all nodes matching tei_element,
        * and reaturns a dictionary containing
        ** the name of the searched element: 'tei_element',
        ** the number of hits: 'nr_of_hits', 
        ** and a list of the found element (as lxml element objects)
        """
        
        result = {'tei_element': tei_element}
        result['hits'] = self.tree.xpath('//tei:text//tei:{}'.format(tei_element), namespaces=self.ns_tei)
        result['nr_of_hits'] = len(result['hits'])
        return result
    
    def add_ids(self, tei_element='placeName', id_prefix='some', export=False, export_file="updated"):
        
        """ reads an tei-xml document
        * looks for tei_elements,
        * adds generic @ref (hashed text-node),
        * and returns a tuple containing 
        ** a list of elements,
        ** and the updated xml-tree object.
        """
        
        hits = self.find_elements(tei_element)['hits']
        ids = []
        for x in hits:
            if x.text is None:
                break
            if x.attrib['ref'] is not None:
                ids.append({'text': x.text, 'ref': x.attrib['ref'], 'node': x})                
            else:
                ref = hashlib.md5(x.text.encode('utf-8')).hexdigest()
                x.attrib['ref'] = "#{}_{}".format(id_prefix, ref)
                ids.append({'text': x.text, 'ref': x.attrib['ref'], 'node': x})
        if export:
            file = "{}.xml".format(export_file)
            with open(file, 'wb') as f:
                f.write(ET.tostring(self.tree, pretty_print=True,encoding="UTF-8"))
        return ids, self.tree
        
    def create_index(self, tei_element='placeName', id_prefix='place'):
        
        """ takes a list of elements and transforms them into an index-file"""
        
        nodes = self.add_ids(tei_element, id_prefix)[0]
        places = self.get_places_elements(nodes)
        list_place = ET.Element("listPlace")
        for x in places:
            list_place.append(x)
        new_doc = ET.fromstring(templates.tei_document)
        body = new_doc.xpath('//tei:body', namespaces=self.ns_tei)[0]
        body.append(list_place)      
        return new_doc
    
    def export_index(self, tei_element='placeName', id_prefix='place', export_file='index'):
        new_doc = self.create_index(tei_element, id_prefix)
        file = "{}.xml".format(export_file)
        with open(file, 'wb') as f:
            f.write(ET.tostring(new_doc, pretty_print=True, encoding="UTF-8"))
        return "file stored as {}".format(file)

In [81]:
test = TeiReader('data/output/new.xml')

In [83]:
hits = test.add_ids(export=True)

In [59]:
ET.tostring(hits)

b'<TEI xmlns="http://www.tei-c.org/ns/1.0">\n  <teiHeader>\n      <fileDesc>\n         <titleStmt>\n            <title/>\n         </titleStmt>\n         <publicationStmt>\n            <p/>\n         </publicationStmt>\n         <sourceDesc>\n            <p/>\n         </sourceDesc>\n      </fileDesc>\n  </teiHeader>\n  <text>\n      <body>\n      <listPlace><place xml:id="some_273c6fe7cd0300c46e98e05cc3414e33"><placeName>Quaipark</placeName></place><place xml:id="some_160b5b00de22ae4c96a1e90460b8c441"><placeName>Mallnitz</placeName></place><place xml:id="some_f1efe1f6313c4676413f70db8feb79f6"><placeName>Spittal</placeName></place><place xml:id="some_8d52cda5b0caaafb17915b780d894888"><placeName>Gastein</placeName></place><place xml:id="some_6924ee63c56474bfac826b3b7e0714cd"><placeName>Heiligenblut</placeName></place><place xml:id="some_6fc6f10422bfe80ddcba23b4f7f080dc"><placeName>Hietzing</placeName></place><place xml:id="some_36cb84b200c47d2c7a7e8dbbe3c1a595"><placeName>Wiednerth.</pl